![PK](https://uploads-ssl.webflow.com/61d1def902a2a8e195cf22d4/65a535e7e2fc9482439bab3e_svgomg-Website%20Poppy%20Logo%20V16%20transparent%20(450%20x%20150%20px).svg)

Poppy Kids Pediatric Dentistry, Novato, CA specializes in creating happy and healthy smiles for infants, children, and teens in a supportive and friendly environment. Dr. Andrea Aduna and the Poppy Kids team strive to provide each patient with individualized, high-quality dental care. As a specialized boutique practice, we believe in building relationships with each patient that promote positivity and lifelong oral health.

Visit us @ [Poppy Kids Pediatric Dentistry](www.poppykidsdental.com/)


In [ ]:
USE POPPY;

In [ ]:
Select count(*) from POPPY.ANALYTICS.GOOGLE_REVIEWS_V;

In [ ]:
select column_name, is_nullable, data_type from information_schema.columns where table_schema = 'ANALYTICS' and Table_name = 'GOOGLE_REVIEWS_V' order by ordinal_position;

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly as px

# Snowpark
from snowflake.snowpark.context import get_active_session
import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import when, date_part 

# Cortex Functions
import snowflake.cortex  as cortex

session = get_active_session()


We can preview the reviews we have in the analytics view using Snowpark Dataframe.

In [ ]:
reviews2024_df = session.table('POPPY.ANALYTICS.GOOGLE_REVIEWS_V') \
             .filter(date_part("year", F.col('REVIEW_DATETIME_PT')) == 2024)
reviews2024_df.show(5)

In [ ]:
# Execute SQL query to get the data
query = """
SELECT YEAR(REVIEW_DATETIME_PT) as year, COUNT(REVIEW_ID) as review_count
FROM Poppy.analytics.google_reviews_v
GROUP BY YEAR(REVIEW_DATETIME_PT)
ORDER BY year
"""

# Execute the query and fetch results into a pandas DataFrame
df = session.sql(query).to_pandas()

# Set the 'YEAR' column as the index
df.set_index('YEAR', inplace=True)

# Add a title to the chart
st.subheader('Number of Reviews per Year')

# Create the bar chart using Streamlit
st.bar_chart(
    data=df,
    y='REVIEW_COUNT',
    width=0,
    height=400,
    use_container_width=True,
    color='#f97212'
)


In [ ]:
SELECT 
    YEAR(REVIEW_DATETIME_PT) as year,
    HOUR(REVIEW_DATETIME_PT) as hour,
    COUNT(REVIEW_ID) as review_count
FROM Poppy.analytics.google_reviews_v
GROUP BY YEAR(REVIEW_DATETIME_PT), HOUR(REVIEW_DATETIME_PT)
HAVING COUNT(REVIEW_ID) > 1
ORDER BY year, hour

In [ ]:
query = """
SELECT 
    YEAR(REVIEW_DATETIME_PT) as year,
    HOUR(REVIEW_DATETIME_PT) as hour,
    COUNT(REVIEW_ID) as review_count
FROM Poppy.analytics.google_reviews_v
GROUP BY YEAR(REVIEW_DATETIME_PT), HOUR(REVIEW_DATETIME_PT)
HAVING COUNT(REVIEW_ID) > 2
ORDER BY year, hour
"""
# Execute the query and fetch results into a pandas DataFrame
df = session.sql(query).to_pandas()

# Pivot the DataFrame to have years as rows and hours as columns
df_pivot = df.pivot(index='YEAR', columns='HOUR', values='REVIEW_COUNT').fillna(0)

# # Ensure all hours (0-23) are present, fill with 0 if missing
# for hour in range(24):
#     if hour not in df_pivot.columns:
#         df_pivot[hour] = 0

# Sort the columns (hours)
df_pivot = df_pivot.sort_index(axis=1)

In [ ]:
import altair as alt

# Melt the pivot table to long format for Altair
df_melt = df_pivot.reset_index().melt(id_vars=['YEAR'], var_name='HOUR', value_name='REVIEW_COUNT')

# Create the stacked bar chart
chart = alt.Chart(df_melt).mark_bar().encode(
    x=alt.X('YEAR:O', title='Year'),
    y=alt.Y('REVIEW_COUNT:Q', title='Number of Reviews'),
    color=alt.Color('HOUR:N', scale=alt.Scale(scheme='viridis'), title='Hour of Day'),
    tooltip=['YEAR', 'HOUR', 'REVIEW_COUNT']
).properties(
    title='Number of Reviews per Hour per Year',
    width=600,
    height=400
)

# Display the chart using Streamlit
st.altair_chart(chart, use_container_width=True)

### Snowflake Cortex

Snowflake Cortex is an intelligent, fully managed service that hosts and serves top-performing LLMs and vector functions which quickly and securely process text and build AI applications contextualized with  data stored in unstructured and semi-structured formats. We can access it via serverless SQL / Python functions. 
The available capabilities include:
- Perform complex reasoning tasks with your text data using COMPLETE.
- Extract information from unstructured or semi-structured data using EXTRACT_ANSWER.
- Automatically get a synopsis of text with SUMMARIZE.
- Detect the mood or tone of text using SENTIMENT.
- Translate documents to other languages using TRANSLATE.




In the next cell, we leverage **Translate** - one of the **Snowflake Cortex specialised LLM functions** are available in Snowpark ML, to translate the multilingual reviews to english to enable easier analysis for folks who don't speak the language the original review is in.

In [ ]:
reviews_df = session.table('POPPY.ANALYTICS.GOOGLE_REVIEWS_V') 
# Order by REVIEW_DATETIME_PT in descending order and show
reviews_df.orderBy(F.col("REVIEW_DATETIME_PT").desc()).show(3)

In [ ]:
# Understand the sentiment of customer review using Cortex Sentiment
reviews_df = reviews_df.withColumn('SENTIMENT', cortex.Sentiment(F.col('REVIEW_TEXT')))

reviews_df.select(["REVIEW_TEXT","SENTIMENT"]).show(3)

In [ ]:
# Import necessary functions
from snowflake.snowpark.functions import median

# Apply sentiment analysis
reviews_df = reviews_df.withColumn('SENTIMENT', cortex.Sentiment(F.col('REVIEW_TEXT')))

# Calculate median sentiment
median_sentiment = reviews_df.select(median('SENTIMENT').alias('MEDIAN_SENTIMENT')).collect()[0]['MEDIAN_SENTIMENT']

# Display median sentiment
print(f"Median Sentiment: {median_sentiment}")

# Show a sample of reviews with their sentiments
reviews_df.select(["REVIEW_TEXT", "SENTIMENT"]).show(3)

# Optionally, show distribution of sentiments
sentiment_distribution = reviews_df.groupBy('SENTIMENT').count().orderBy('SENTIMENT')
sentiment_distribution.show()

In [ ]:
# Prompt to understand sentiment for different categories mentioned in the customer review
# We employ one shot incontext learning to inform LLM
prompt = """[INST]
### 
You are analyzing pediatric dentistry practice customer reviews to understand what a given review says about different relevant categories like \
quality of service, practice amenities, staff, overall experience, price, ambience, \
hygiene standards etc and if sentiment is negative,positive or neutral for that category. \
Only answer in a single valid JSON containing "category", "sentiment" and "details". \
Make sure there is no additional text and not mention categories in answer which are not \
talked in the review. \
Get category based sentiment for the following customer review:
"This pediatric dentistry practice offers a disappointing experience. \
The staff is not friendly and the services are subpar. They are not open at the times I prefer \
and there are no open slots when we need. Additionally, the value for money is not worth it. To top \
it all off, the service provided at this pediatric dentistry practice is less than pleasant, adding to the overall \
negative dental experience for kids. Tried sending an email to complain but no reply." 
Answer : [{     "category": "services",     "sentiment": "negative",    "details": "subpar quality"   }, \
{     "category": "staff",     "sentiment": "negative",     "details": "not friendly"   },   \
{     "category": "price",     "sentiment": "negative",     "details": "not worth the money"   },   \
{     "category": "experience",     "sentiment": "negative",     "details": "disappointing experience"   },   \
{     "category": "appointmnet scheduling",     "sentiment": "negative",     "details": "no open slots when we need"   },   \
{     "category": "patient support",     "sentiment": "negative",     "details": "no reply to complaint email"   } ].
Get category based sentiment for the follwoing customer review:
###"""

# Ask Cortex Complete and create a new column
review_df = reviews_df.withColumn('CATEGORY_SENTIMENT', cortex.Complete('mixtral-8x7b', \
                                                            F.concat(F.lit(prompt), \
                                                                     F.col('REVIEW_TEXT'), \
                                                                     F.lit("""Answer:[/INST]"""))))
review_df.select(["REVIEW_TEXT","CATEGORY_SENTIMENT"]).show(1)

In [ ]:
# Aggregrate the 100 most negative reviews for tasty bytes
reviews_agg_ = reviews_df.order_by(F.col('SENTIMENT')).select(F.col('REVIEW_TEXT')).first(100)

reviews_agg_str = ''.join(map(str,reviews_agg_))

# Prompt to summarize the three top issues flagged in the aggregated reviews
prompt = """[INST]###Summarize the issues mentioned in following aggregated poppy kids pediatric dentistry customer reviews with three \
concise bullet points under 50 words each such that each bullet point also has a heading along with \
recommendations to remedy those issues.###""" + reviews_agg_str + """[/INST]"""

# Answer from Cortex Complete
print(cortex.Complete('mistral-large', prompt))

In [ ]:
from wordcloud import WordCloud

# Query to get review text
df = session.table('POPPY.ANALYTICS.GOOGLE_REVIEWS_V').select('REVIEW_TEXT')

# Convert to pandas DataFrame
pdf = df.to_pandas()

# Remove null values and ensure all values are strings
pdf = pdf.dropna(subset=['REVIEW_TEXT'])
pdf['REVIEW_TEXT'] = pdf['REVIEW_TEXT'].astype(str)

# Clean text: convert to lowercase, remove punctuation and numbers
pdf['REVIEW_TEXT'] = pdf['REVIEW_TEXT'].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\d+', '', regex=True)

# Combine all reviews into a single string
text = ' '.join(pdf['REVIEW_TEXT'])

# List of stopwords (you can expand this list)
stopwords = set(['the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of',\
                 'with', 'by', 'andrea', 'dr', 'made', 'his', 'would', 'from', 'have', \
                'that', 'go', 'took', 'such', 'really', 'being', 'i', 'so', 'as', 'here', 'are', \
                'just', 'what', 'my', 'she was', 'was', 'this', 'it', 'your', 'out', 'they', 'we', 'about', \
                'him', 'found', 'our', 'he', 'she', 'there', 'can', 'we had', 'is', 'her', 'were'])

# Create WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords, max_words=100).generate(text)

# Streamlit app
st.title('Review Text Word Cloud')

# Display the word cloud directly
st.image(wordcloud.to_array())

st.write('This word cloud shows the most frequent words in the reviews. Common words have been removed.')

# Display the number of reviews processed
st.write(f"Number of reviews processed: {len(pdf)}")

# Let's recreate Wordcloud using mistral-large. 
### So long nltk and stop words.

In [ ]:
# Get all reviews
reviews_df = session.table('POPPY.ANALYTICS.GOOGLE_REVIEWS_V').select('REVIEW_TEXT')
reviews_agg_ = reviews_df.collect()
reviews_agg_str = ' '.join(map(lambda x: str(x[0]), reviews_agg_))

# Prompt to generate keywords for word cloud
prompt = """[INST]Analyze the following aggregated customer reviews for Poppy Kids Pediatric Dentistry. 
Generate a list of 100 key words or short phrases that best represent the content and sentiment of these reviews. 
These words will be used to create a word cloud, so focus on meaningful and representative terms. 
Separate each word or phrase with a comma.

Reviews:
""" + reviews_agg_str + """

Provide only the list of words or phrases in your response, without any additional text.[/INST]"""

# Get response from Cortex Complete
llm_response = cortex.Complete('mistral-large', prompt)



In [ ]:
print(llm_response)

In [ ]:
from wordcloud import WordCloud

# Create WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=100).generate(llm_response)

# Streamlit app
st.title('Poppy Kids Pediatric Dentistry Review Word Cloud')

# Display the word cloud
st.image(wordcloud.to_array())

st.write('This word cloud shows key terms from all reviews, as analyzed by an AI language model.')

# Display the number of reviews processed
st.write(f"Number of reviews processed: {len(reviews_agg_)}")